In [1]:
# import all necessary packages
import pandas as pd
import numpy as np
import json
import datetime
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib as mlp
import matplotlib.pyplot as plt
#from geopy.distance import great_circle
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from datetime import datetime

# import dataset
dfPhiladelphia = pd.read_csv("data/tmp/dfPhiladelphia.csv.zip")
dfPhiladelphia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 733441 entries, 0 to 733440
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   start_date               733441 non-null  object 
 1   end_date                 733441 non-null  object 
 2   start_time_hourly        733441 non-null  int64  
 3   end_time_hourly          733441 non-null  int64  
 4   start_station_id         733441 non-null  int64  
 5   end_station_id           733441 non-null  int64  
 6   start_latitude           733441 non-null  float64
 7   start_longitude          733441 non-null  float64
 8   end_latitude             733441 non-null  float64
 9   end_longitude            733441 non-null  float64
 10  min_temp                 733441 non-null  float64
 11  max_temp                 733441 non-null  float64
 12  isPrecip                 733441 non-null  bool   
 13  user_type                733441 non-null  object 
 14  dayO

In [ ]:
X = dfPhiladelphia[[""]]

In [ ]:
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

def plot_dendrogram(model, **kwargs):

    # Children of hierarchical clustering
    children = model.children_

    # Distances between each pair of children
    # Since we don't have this information, we can use a uniform one for plotting
    distance = np.arange(children.shape[0])

    # The number of observations contained in each cluster level
    no_of_observations = np.arange(2, children.shape[0]+2)

    # Create linkage matrix and then plot the dendrogram
    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)


agglo = AgglomerativeClustering(n_clusters=5) #The number of clusters to find
y_pred_agglo = agglo.fit_predict(X_scaled)

plt.figure(figsize=(15,10))
plt.title('Hierarchical Clustering Dendrogram')
plot_dendrogram(agglo, labels=agglo.labels_)
plt.ylabel("Distance")
plt.show()

# 3. Cluster Analysis

We use a **Cluster Analysis** in order to identify clusters of 
 - *trip types* and/or
 - *customer types*
 
based on the bike rental demand patterns.

Additionally we are interested, whether we can *label these clusters* and *cluster the locations* based on their demand pattern.

## 3.1 Data Prep and Scaling

 - Hier die Features aus Task 1 aufführen, die wir für das Clustering benötigen

## 3.2 Principal Component Analysis (PCA)
- for dimensionality reduction

## PAIRPLOT

In [ ]:
#sns.pairplot(data=, hue="")

## 3.3 Clustering with Dendrogram

In [ ]:
#from scipy.cluster.hierarchy import dendrogram
#from sklearn.cluster import AgglomerativeClustering
#
#def plot_dendrogram(model, **kwargs):
#
#    # Children of hierarchical clustering
#    children = model.children_
#
#    # Distances between each pair of children
#    # Since we don't have this information, we can use a uniform one for plotting
#    distance = np.arange(children.shape[0])
#
#    # The number of observations contained in each cluster level
#    no_of_observations = np.arange(2, children.shape[0]+2)
#
#    # Create linkage matrix and then plot the dendrogram
#    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)
#
#    # Plot the corresponding dendrogram
#    dendrogram(linkage_matrix, **kwargs)
#
#
#agglo = AgglomerativeClustering(n_clusters=3) #The number of clusters to find
#y_pred_agglo = agglo.fit_predict(X_scaled)
#
#plt.figure(figsize=(15,10))
#plt.title('Hierarchical Clustering Dendrogram')
#plot_dendrogram(agglo, labels=agglo.labels_)
#plt.ylabel("Distance")
#plt.show()